<a href="https://colab.research.google.com/github/Mohammed287-lab/ANN/blob/main/olive_leaf_mode2l.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#file save

In [1]:
from google.colab import files

uploaded = files.upload()  # ارفع dataset.zip


Saving archive (13).zip to archive (13).zip


In [2]:
import zipfile
import os

zip_path = "archive (13).zip"   # اسم الملف المرفوع
extract_path = "/content/dataset"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("تم فك الضغط بنجاح")


تم فك الضغط بنجاح


In [3]:
for root, dirs, files in os.walk("/content/dataset"):
    level = root.replace("/content/dataset", "").count(os.sep)
    indent = " " * 4 * level
    print(f"{indent}{os.path.basename(root)}/")


dataset/
    dataset/
        train/
            Healthy/
            aculus_olearius/
            olive_peacock_spot/
        test/
            Healthy/
            aculus_olearius/
            olive_peacock_spot/


#pre

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_size = (224, 224)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(
    rescale=1./255
)

train_generator = train_datagen.flow_from_directory(
    "/content/dataset/dataset/train",
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical"
)

test_generator = test_datagen.flow_from_directory(
    "/content/dataset/dataset/test",
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical"
)


Found 2720 images belonging to 3 classes.
Found 680 images belonging to 3 classes.


#model


In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


In [6]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  # عدد الكلاسات = 3
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,169,347 (42.61 MB)

 Trainable params: 11,169,347 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
epochs = 10

history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=epochs
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 377s 4s/step - accuracy: 0.4749 - loss: 1.1653 - val_accuracy: 0.5279 - val_loss: 0.8580
Epoch 2/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 364s 4s/step - accuracy: 0.6107 - loss: 0.7619 - val_accuracy: 0.4397 - val_loss: 1.0074
Epoch 3/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 369s 4s/step - accuracy: 0.5440 - loss: 0.8926 - val_accuracy: 0.6324 - val_loss: 0.8645
Epoch 4/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 362s 4s/step - accuracy: 0.6417 - loss: 0.7362 - val_accuracy: 0.5779 - val_loss: 0.8196
Epoch 5/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 354s 4s/step - accuracy: 0.6670 - loss: 0.6976 - val_accuracy: 0.6279 - val_loss: 0.7535
Epoch 6/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 366s 4s/step - accuracy: 0.6839 - loss: 0.6650 - val_accuracy: 0.6368 - val_loss: 0.7569
Epoch 7/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 362s 4s/step - accuracy: 0.7104 - loss: 0.6161 - val_accuracy: 0.6029 - val_loss: 0.7483
Epoch 8/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 351s 4s/step - accuracy: 0.7239 - loss: 0.6083 - val_accuracy: 0.5574 - v

In [13]:
import numpy as np
from tensorflow.keras.preprocessing import image

img_path = "/content/dataset/dataset/test/aculus_olearius/A12.JPG"   # corrected path
img = image.load_img(img_path, target_size=img_size)
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0) / 255.0

pred = model.predict(img)
print("Prediction:", pred)
print("Class index: ", pred.argmax())

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 198ms/step
Prediction: [[0.4147842  0.4366102  0.14860564]]
Class index:  1


In [25]:
!pip install ipywidgets


In [54]:
from google.colab import output
output.enable_custom_widget_manager()


In [56]:
import tensorflow as tf
import numpy as np
from PIL import Image
import io

model = tf.keras.models.load_model("my_model.h5")

class_names = ["class1", "class2", "class3"]

In [57]:
import ipywidgets as widgets
from IPython.display import display, clear_output

uploader = widgets.FileUpload(
    accept='image/*',
    multiple=False
)

button = widgets.Button(
    description='Predict',
    button_style='success'
)

output_box = widgets.Output()

display(uploader, button, output_box)


FileUpload(value={}, accept='image/*', description='Upload')

Button(button_style='success', description='Predict', style=ButtonStyle())

Output()

In [50]:
def predict_image(b):
    with output_box:
        clear_output()

        if len(uploader.value) == 0:
            print("Please upload an image")
            return

        file_info = list(uploader.value.values())[0]
        image_bytes = file_info['content']

        img = Image.open(io.BytesIO(image_bytes)).convert("RGB")
        img = img.resize((224, 224))

        img_array = np.array(img) / 255.0
        img_array = np.expand_dims(img_array, axis=0)

        pred = model.predict(img_array)
        class_index = np.argmax(pred)
        confidence = np.max(pred) * 100

        display(img)
        print(f"Prediction: {class_names[class_index]}")
        print(f"Confidence: {confidence:.2f}%")

button.on_click(predict_image)


##save_model

In [24]:
model.save("my_model.h5")